## 機器學習模型訓練與 API 部署
這是我使用keras訓練神經網路的程式碼。<br>
這個模型是關於商品推薦的模型，會根據顧客對電影的評價，推薦給顧客他可能會喜歡的電影<br>
使用的機器學習方法是Embedding與NN神經網路。<br>
**對應於均一，可以推薦適合的課程給學生，達到客製化推薦的目的。**<br>
Flask套件部署我的神經網路模型至網路上的程式碼<br>
部署後的資料，會使用Json的格式傳送到前端，再由前端工程師將其視覺化。<br>

In [6]:
import pandas as pd
df = pd.read_csv("./Data/0913_all_movie_rating").drop(columns = 'Unnamed: 0')

In [7]:
df.head()

,userId,movieId,rating,y,n_ratings,title
0,1,2,3.5,-0.2,22243,Jumanji
1,5,2,3.0,-0.7,22243,Jumanji
2,13,2,3.0,-0.7,22243,Jumanji
3,29,2,3.0,-0.7,22243,Jumanji
4,34,2,3.0,-0.7,22243,Jumanji


In [8]:
import requests
def telegram_bot_sendtext(bot_message):
    
    bot_token = '661544421:AAHuZSMukZmFlhU-npE23CowogP375a0au4'
    bot_chatID = '-225157366'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()


In [11]:
import numpy as np
import keras
import tensorflow as tf


In [12]:
df.head()

,userId,movieId,rating,y,n_ratings,title
0,1,2,3.5,-0.2,22243,Jumanji
1,5,2,3.0,-0.7,22243,Jumanji
2,13,2,3.0,-0.7,22243,Jumanji
3,29,2,3.0,-0.7,22243,Jumanji
4,34,2,3.0,-0.7,22243,Jumanji


In [13]:
Emb_size = 32
movie_r12n = keras.regularizers.l2(1e-6)
user_r12n = keras.regularizers.l2(1e-7)
movie_r12n = None
user_r12n = None

user_input = keras.Input(shape = (1,), name = "user_id")
movie_input = keras.Input(shape = (1,), name = "movie_id")
mean_rate = keras.Input(shape = (1,), name = "mean_rate")
Embedd_user = keras.layers.Embedding(df.userId.max()+10,Emb_size, name="Embedd_user",
                                     input_length=1, embeddings_regularizer= movie_r12n)(user_input)
Embedd_movies = keras.layers.Embedding(df.movieId.max()+10,Emb_size, name="Embedd_movies", 
                                       input_length=1, embeddings_regularizer= user_r12n)(movie_input)
user_dropout = keras.layers.Dropout(0.3, name="Dropout_user")(Embedd_user)
movie_dropout = keras.layers.Dropout(0.3, name="Dropout_movie")(Embedd_movies)
Dot = keras.layers.Dot(2)([user_dropout, movie_dropout])
Flatten = keras.layers.Flatten()(Dot)
model = keras.Model(
    inputs = [user_input, movie_input],
    outputs = Flatten
)

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
movie_id (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
Embedd_user (Embedding)         (None, 1, 32)        4432096     user_id[0][0]                    
__________________________________________________________________________________________________
Embedd_movies (Embedding)       (None, 1, 32)        4200704     movie_id[0][0]                   
____________________________________________________________________________________________

In [14]:
u_train = df.userId
m_train = df.movieId
y_train = df.y

In [42]:
from sklearn.model_selection import train_test_split
u_train, u_test, m_train, m_test, y_train, y_test = train_test_split(df.userId, df.movieId, df.y,
                                                                     test_size= 0.3,
                                                                    random_state = 878)
print("split to:\ntrain: %d\ntest:  %d"%(u_train.shape[0], u_test.shape[0]))

split to:
train: 14000184
test:  6000079


In [15]:

'''import tensorflow as tf
from tensorflow.python import debug as tf_debug
import keras
run_metadata = tf.RunMetadata()
keras.backend.set_session(
    tf_debug.TensorBoardDebugWrapperSession(tf.Session(), "fio:6007"))'''
pass

In [16]:
from keras.callbacks import TensorBoard
import time
from time import time as currentTime
import tensorflow as tf
tensorboard = TensorBoard(log_dir="logs/{}".format(currentTime()))

In [17]:

model.compile(
    # Technical note: when using embedding layers, I highly recommend using one of the optimizers
    # found  in tf.train: https://www.tensorflow.org/api_guides/python/train#Optimizers
    # Passing in a string like 'adam' or 'SGD' will load one of keras's optimizers (found under 
    # tf.keras.optimizers). They seem to be much slower on problems like this, because they
    # don't efficiently handle sparse gradient updates.
    tf.train.AdamOptimizer(0.005),
    loss='MSE',
    metrics=['MAE'],
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
movie_id (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
Embedd_user (Embedding)         (None, 1, 32)        4432096     user_id[0][0]                    
__________________________________________________________________________________________________
Embedd_movies (Embedding)       (None, 1, 32)        4200704     movie_id[0][0]                   
____________________________________________________________________________________________

In [18]:

history = model.fit([u_train, m_train], y_train, batch_size= 100000, epochs=40,
    verbose=1, callbacks = [tensorboard], validation_data=[[u_train, m_train], y_train],
                    validation_freq= 5)
 #


Train on 20000263 samples, validate on 20000263 samples
Epoch 1/40
20000263/20000263 [==============================] - 8s 0us/step - loss: 0.9393 - mean_absolute_error: 0.7503
Epoch 2/40
20000263/20000263 [==============================] - 7s 0us/step - loss: 0.7209 - mean_absolute_error: 0.6507
Epoch 3/40
20000263/20000263 [==============================] - 8s 0us/step - loss: 0.6920 - mean_absolute_error: 0.6378
Epoch 4/40
20000263/20000263 [==============================] - 8s 0us/step - loss: 0.6779 - mean_absolute_error: 0.6317
Epoch 5/40
20000263/20000263 [==============================] - 12s 1us/step - loss: 0.6687 - mean_absolute_error: 0.6275 - val_loss: 0.5833 - val_mean_absolute_error: 0.5849
Epoch 6/40
20000263/20000263 [==============================] - 7s 0us/step - loss: 0.6620 - mean_absolute_error: 0.6246
Epoch 7/40
20000263/20000263 [==============================] - 8s 0us/step - loss: 0.6570 - mean_absolute_error: 0.6224
Epoch 8/40
20000263/20000263 [=============

In [ ]:
result = model.predict([u_train, m_train]).reshape(-1) - y_train
train_mean=abs(result).mean()
train_std = result.std()
del result
result = model.predict([u_test, m_test]).reshape(-1) - y_test
test_mean = abs(result).mean()
test_std = result.std()
del result

In [ ]:
result = "結果出來了，訓練結果訓練資料:\nabs(loss).mean() = "+ str(train_mean) + " \n標準差 = " + str(train_std)+"\n\n測試資料:\nabs(loss).mean() = "+ str(test_mean) + " \n標準差 = " + str(test_std)

telegram_bot_sendtext(result)

print(result)

In [ ]:
import numpy as np
import keras
import tensorflow as tf
import json

#匯入模型
model = keras.models.load_model("./0903_size32.h5")
model.trainable = False
model.compile(
    # Technical note: when using embedding layers, I highly recommend using one of the optimizers
    # found  in tf.train: https://www.tensorflow.org/api_guides/python/train#Optimizers
    # Passing in a string like 'adam' or 'SGD' will load one of keras's optimizers (found under 
    # tf.keras.optimizers). They seem to be much slower on problems like this, because they
    # don't efficiently handle sparse gradient updates.
    tf.train.AdamOptimizer(0.005),
    loss='MSE',
    metrics=['MAE'],
)
#model.summary()
temp = model.predict([np.array([3]).reshape(-1,1), np.array([3]).reshape(-1,1)])
del temp


embedd_movies = model.get_layer(name = "Embedd_movies")

(w,) = embedd_movies.get_weights()
movie_embedding_size = w.shape[1]

from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

#匯入資料
import pandas as pd
movies = pd.read_csv("/home/fio/Python/movielens_20m/Data/movie_pro.csv")
movies = movies[movies.n_rating>1000]

def munge_title(title):
    l = title.rfind('(') + 1
    year = int(title[l:l+4])
    i = title.rfind(' (')
    if i != -1:
        title = title[:i]
    for suff_word in ['The', 'A', 'An']:
        suffix = ', {}'.format(suff_word)
        if title.endswith(suffix):
            title = suff_word + ' ' + title[:-len(suffix)]
    return title+' ('+str(year)+')'

movies['title'] = movies['title'].map(munge_title)

moviename = movies.title

movies = movies[movies.n_rating>1000]

movie_id_list = np.array(movies.movieId)
movie_id_len = len(movie_id_list)
title_list = movies['title'].copy()
title_list_total = pd.DataFrame(data = title_list)
title_list_total['movieId'] = np.array(movies.movieId)

#get similar
kv = WordEmbeddingsKeyedVectors(movie_embedding_size)
kv.add(
    movies['title'].values,
    w[movies.movieId]
)

#kv.most_similar(name)



#定義API

from flask import Flask, request, abort
from flask_cors import CORS
app = Flask(__name__)
CORS(app)

from waitress import serve

@app.route("/callback")
def callback():
    return 'OK'

@app.route("/api/gs/<string:name>")
def similar_first(name):
    try: name = np.array(moviename[moviename.str.contains(name)])[0]
    except IndexError:return "Movie not found."
    #print("Most similar with %s:"%(name))
    string = 'According to the rates by audience.<br>The movie %s is more similar to:<br><br>'%(name)
    try:
        for i,j in kv.most_similar(name):
            string+="%s, %f <br/>"%(i,j)
    except KeyError: 
        print(" Error Key : %s"%(name))
        return "Key Error, Try another word."
    return string

@app.route("/api/gsj_title/<string:title>")
def similar_title_json(title):
    nums = 5
    movie_title = np.array(moviename[moviename.str.contains(title)])[0]
    movieid = list(movies[movies.title == movie_title].movieId)[0]
    result = kv.most_similar(movie_title)
    target_movie_title_list = []
    movie_similar_rate = []
    target_movie_id_list = []
    for i in range(nums):
        target = result[i]
        target_movie_title_list.append(target[0])
        movie_similar_rate.append(target[1])
    out = {
        "input_movieId": movieid,
        "input_movie_title" : str(movie_title),
        "similar_movie_title_list" : target_movie_title_list,
        "similar_movie_similar_rate" : movie_similar_rate
    }
    return json.dumps(out)

@app.route("/api/gsj_2_title/<string:title>")
def similar_2_title_json(title):
    title = title.split("<>")
    nums = 5
    movie_title = []
    movieid = []
    for i in title:
        movie_title.append(np.array(moviename[moviename.str.contains(i)])[0])
        movieid.append(list(movies[movies.title == movie_title[-1]].movieId)[0])
    result = kv.most_similar(movie_title)
    target_movie_title_list = []
    movie_similar_rate = []
    target_movie_id_list = []
    for i in range(nums):
        target = result[i]
        target_movie_title_list.append(target[0])
        movie_similar_rate.append(target[1])
    out = {
        "input_movieId": movieid,
        "input_movie_title" : str(movie_title),
        "similar_movie_title_list" : target_movie_title_list,
        "similar_movie_similar_rate" : movie_similar_rate
    }
    return json.dumps(out)


@app.route("/api/gsj_id/<int:movieid>")
def similar_id_json(movieid):
    nums = 5
    movie_title = title_list_total[title_list_total.movieId == movieid].title
    result = kv.most_similar(movie_title)
    target_movie_title_list = []
    movie_similar_rate = []
    target_movie_id_list = []
    for i in range(nums):
        target = result[i]
        target_movie_title_list.append(target[0])
        movie_similar_rate.append(target[1])
    out = {
        "input_movieId": movieid,
        "input_movie_title" : str(list(movie_title)[0]),
        "similar_movie_title_list" : target_movie_title_list,
        "similar_movie_similar_rate" : movie_similar_rate
    }
    return json.dumps(out)

@app.route("/api/rm/<int:userId>")
def recommand(userId):
    num = 5
    ids = [userId]*movie_id_len
    result = model.predict([ids, movie_id_list])
    title_list = title_list_total.copy()
    title_list['predict'] = result+3.7 
    title_list = title_list.sort_values("predict", ascending=False).head(num)
    rating_list = list(title_list.predict)
    title_list = list(title_list.title)
    string = ("User %d will most likely to like:<br/>"%(ids[0]))
    for i in range(len(title_list)):
        string+=(" %.1f    %s <br>"%(rating_list[i], title_list[i]))
    return string

@app.route("/api/rmj/<int:userId>")
def recommand_json(userId):
    num = 5
    ids = [userId]*movie_id_len
    result = model.predict([ids, movie_id_list])
    title_list = title_list_total.copy()
    title_list['predict'] = result+3.7 
    title_list = title_list.sort_values("predict", ascending=False).head(num)
    title_list.head()
    rating_list = list(title_list.predict)
    movie_title_list = list(title_list.title)
    id_list = list(title_list.movieId)
    out_title_list = []
    out_movieId_list = []
    out_movie_rating = []
    for i in range(len(movie_title_list)):
        out_title_list.append(movie_title_list[i])
        out_movieId_list.append(id_list[i])
        out_movie_rating.append(rating_list[i])
    out = {'error_msg':"",
           'userId': ids[0],
           'recommend_movie_id': out_movieId_list,
           'recommend_movie_title': out_title_list,
           'rating_sort': out_movie_rating}
    return json.dumps(out)

@app.route("/api/sf/<string:inputs>")
def predict_s(inputs):
    user = inputs.split("<>")[0]
    movie_name = inputs.split("<>")[1:]
    print(inputs)
    user = [user] * len(movie_name)
    movie_ids = []
    movie_name_list = []
    for i in range(len(movie_name)):
        try: 
            temp_df = movies[movies.title.str.contains(movie_name[i])].head(1)
        except IndexError:
            return "Movie not found."
        try: movie_name_list.append(str(temp_df.title)[3:].replace("    ","").replace("\nName: title, dtype: object",""))
        except TypeError:
            return "Movie not found."
        try: movie_ids.append(int(temp_df.movieId))
        except TypeError:
            return "Movie not found."
    #print(movie_name)
    result = model.predict([user, movie_ids]) + 3.7
    string = "User %d to below movies rating prediction is:<br>"%(int(user[0]))
    for i in range(len(movie_name_list)):
        string += "%.2f - %s<br>"%(result[i], movie_name_list[i])
    return string

@app.route("/api/demo_movie")
def get_demo_movie():
    out_call_df = pd.read_csv("/home/fio/Python/movielens_20m/Data/out_call_df.csv")[['title', 'movieId', 'n_rating', 'genres', 'year']]
    out_title = list(out_call_df.title)
    out_movieId = list(out_call_df.movieId)
    out_rate_times = list(out_call_df.n_rating)
    out_film_year = list(out_call_df.year)
    out_genres = []
    for i in out_call_df.genres:
        out_genres.append(list(i.split("|")))
    out = {
        "error_msg": "",
        "title": out_title,
        "movieId": out_movieId,
        "rate_times": out_rate_times,
        "genres": out_genres,
        "film_year": out_film_year,
    }
    return json.dumps(out)
 
@app.route("/api/demo_user")
def get_demo_user():
    out = {
        "error_msg": "",
        "userId": list(range(1,26,1)),
    }
    return json.dumps(out)
    
#啟動server
import sys 
print(sys.version)
if __name__ == "__main__":
    serve(app, host = '0.0.0.0', port = 12348)